In [1]:
# !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
# !python rapidsai-csp-utils/colab/pip-install.py

# %load_ext cudf.pandas

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import multiprocessing as mp
import gc
import datetime
from sklearn.preprocessing import LabelEncoder
import calendar
from scipy.sparse import csr_matrix,hstack
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from tqdm import tqdm
import pickle
import sys
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [3]:
pd

<module 'pandas' from '/usr/local/lib/python3.10/dist-packages/pandas/__init__.py'>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
!cp /content/gdrive/MyDrive/PredictiveAnalyticsData/utility/generate_submission_file.py .
from generate_submission_file import generate_submission

Mounted at /content/gdrive
Training Data Shape: (58327370, 43)
Testing Data Shape: (853720, 43)
Prediction Data Shape: (853720, 43)


In [6]:
def separate_features_and_parameters():
  df = pd.read_pickle("/content/gdrive/MyDrive/PredictiveAnalyticsData/output_data/final_train.pkl")
  # train_df = df.drop(columns=['id'], axis=1)
  train_df = df.copy()
  print(f"Training Data Shape: {train_df.shape}")
  y_train = train_df['sales']
  X_train = train_df.drop(columns=['sales'], axis=1)
  X_train[['id','sold_lag_7', 'sold_lag_14', 'sold_lag_21', 'sold_lag_28', 'sold_lag_35', 'sold_lag_42', 'sold_lag_49', 'sold_lag_56', 'roll_mean_7', 'roll_mean_14', 'roll_mean_28']] = X_train[['id', 'sold_lag_7', 'sold_lag_14', 'sold_lag_21', 'sold_lag_28', 'sold_lag_35', 'sold_lag_42', 'sold_lag_49', 'sold_lag_56', 'roll_mean_7', 'roll_mean_14', 'roll_mean_28']].fillna(0.0)

  df = pd.read_pickle("/content/gdrive/MyDrive/PredictiveAnalyticsData/output_data/final_test.pkl")
  # test_df = df.drop(columns=['id'], axis=1)
  test_df = df.copy()
  print(f"Testing Data Shape: {test_df.shape}")
  y_test = test_df['sales']
  X_test = test_df.drop(columns=['sales'], axis=1)
  X_test[['id','sold_lag_7', 'sold_lag_14', 'sold_lag_21', 'sold_lag_28', 'sold_lag_35', 'sold_lag_42', 'sold_lag_49', 'sold_lag_56', 'roll_mean_7', 'roll_mean_14', 'roll_mean_28']] = X_test[['id', 'sold_lag_7', 'sold_lag_14', 'sold_lag_21', 'sold_lag_28', 'sold_lag_35', 'sold_lag_42', 'sold_lag_49', 'sold_lag_56', 'roll_mean_7', 'roll_mean_14', 'roll_mean_28']].fillna(0.0)

  df = pd.read_pickle("/content/gdrive/MyDrive/PredictiveAnalyticsData/output_data/final_prediction_data.pkl")
  # prediction_df = df.drop(columns=['id'], axis=1)
  prediction_df = df.copy()
  print(f"Prediction Data Shape: {prediction_df.shape}")
  y_prediction = prediction_df['sales']
  X_prediction = prediction_df.drop(columns=['sales'], axis=1)
  X_prediction[['id','sold_lag_7', 'sold_lag_14', 'sold_lag_21', 'sold_lag_28', 'sold_lag_35', 'sold_lag_42', 'sold_lag_49', 'sold_lag_56', 'roll_mean_7', 'roll_mean_14', 'roll_mean_28']] = X_prediction[['id', 'sold_lag_7', 'sold_lag_14', 'sold_lag_21', 'sold_lag_28', 'sold_lag_35', 'sold_lag_42', 'sold_lag_49', 'sold_lag_56', 'roll_mean_7', 'roll_mean_14', 'roll_mean_28']].fillna(0.0)

  del df

  return X_train, y_train, X_test, y_test, X_prediction, y_prediction

In [7]:
X_train, y_train, X_test, y_test, X_prediction, y_prediction = separate_features_and_parameters()

Training Data Shape: (58327370, 43)
Testing Data Shape: (853720, 43)
Prediction Data Shape: (853720, 43)


In [8]:
# for col in ["sold_lag_7", "sold_lag_14", "sold_lag_21", "sold_lag_28", "sold_lag_35", "sold_lag_42", "sold_lag_49", "sold_lag_56", "roll_mean_7", "roll_mean_14", "roll_mean_28"]:
#   X_train[col] = X_train[col].fillna(0.0)

In [9]:
# Load your dataset (replace 'your_data.csv' with your actual file)

# Assuming 'daily_sales' is the column containing daily unit sales
daily_sales = y_train.values.reshape(-1, 1)

# Extract other features
features = X_train.values

# Normalize the data
scaler_sales = MinMaxScaler(feature_range=(0, 1)).fit(daily_sales)
scaler_features = MinMaxScaler(feature_range=(0, 1)).fit(features)

# Function to create sequences for LSTM
def create_sequences(data, n_steps):
    for i in range(len(data) - n_steps):
        X = data[i:i + n_steps, :-1].reshape(1, n_steps, -1)
        y = data[i + n_steps, -1].reshape(1, 1)
        return X, y

# Define the number of time steps (adjust as needed)
n_steps = 7

In [10]:
# Create generator for sequences
X, y = create_sequences(np.concatenate((scaler_features.transform(features), scaler_sales.transform(daily_sales)), axis=1), n_steps)



In [13]:
y

array([[0.01572739]])

In [11]:
# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(n_steps, features.shape[1])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

# Make predictions
predictions_scaled = model.predict(X_test)
predictions = scaler_sales.inverse_transform(predictions_scaled.reshape(-1, 1))

# Inverse transform actual values for comparison
y_test_actual = scaler_sales.inverse_transform(y_test.reshape(-1, 1))

# Evaluate the model
mse = mean_squared_error(y_test_actual, predictions)
print(f'Mean Squared Error: {mse}')

# Plot the results
plt.plot(y_test_actual, label='Actual')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.show()

ValueError: ignored